本次以英雄联盟对局胜负预测任务为基础，要求实现决策树算法相关细节，加深对算法的理解，并了解做机器学习任务的大致流程。

### 任务介绍
英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。

### 导入工具包
pandas是数据分析和处理常用的工具包，非常适合处理行列表格数据。numpy是数学运算工具包，支持高效的矩阵、向量运算。sklearn是机器学习常用工具包，包括了一些已经实现好的简单模型和一些常用数据处理方法、评价指标等函数。

In [66]:
from collections import Counter
import pandas as pd # 数据处理
import numpy as np # 数学运算
from sklearn.model_selection import train_test_split, cross_validate # 划分数据集函数
from sklearn.metrics import accuracy_score # 准确率函数
RANDOM_SEED = 2020 # 固定随机种子

### 读入数据

In [67]:
csv_data = './data/high_diamond_ranked_10min.csv' # 数据路径
data_df = pd.read_csv(csv_data, sep=',') # 读入csv文件为pandas的DataFrame
data_df = data_df.drop(columns='gameId') # 舍去对局标号列

In [68]:
data_df

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,0,28,2,1,9,6,11,0,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,0,12,1,0,5,5,5,0,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,0,15,0,0,7,11,4,1,1,0,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,0,43,1,0,4,5,5,1,0,1,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,0,75,4,0,6,6,6,0,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,1,17,2,1,7,4,5,1,1,0,...,0,15246,6.8,16498,229,34,-2519,-2469,22.9,1524.6
9875,1,54,0,0,6,4,8,1,1,0,...,0,15456,7.0,18367,206,56,-782,-888,20.6,1545.6
9876,0,23,1,0,6,7,5,0,0,0,...,0,18319,7.4,19909,261,60,2416,1877,26.1,1831.9
9877,0,14,4,1,2,3,3,1,1,0,...,0,15298,7.2,18314,247,40,839,1085,24.7,1529.8


###  数据概览
对于一个机器学习问题，在拿到任务和数据后，首先需要观察数据的情况，比如我们可以通过`.iloc[0]`取出数据的第一行并输出。不难看出每个特征都存成了float64浮点数，该对局蓝色方开局10分钟有小优势。同时也可以发现有些特征列是重复冗余的，比如blueGoldDiff表示蓝色队金币优势，redGoldDiff表示红色方金币优势，这两个特征是完全对称的互为相反数。blueCSPerMin是蓝色方每分钟击杀小兵数，它乘10就是10分钟所有小兵击杀数blueTotalMinionsKilled。在之后的特征处理过程中可以考虑去除这些冗余特征。
另外，pandas有非常方便的`describe()`函数，可以直接通过DataFrame进行调用，可以展示每一列数据的一些统计信息，对数据分布情况有大致了解，比如blueKills蓝色方击杀英雄数在前十分钟的平均数是6.14、方差为2.93，中位数是6，百分之五十以上的对局中该特征在4-8之间，等等。

blueWins(target): The target column. 1 if the blue team has won, 0 otherwise.

1. blueWardsPlaced: Number of warding totems placed by the blue team on the map
2. blueWardsDestroyed: Number of enemy warding totems the blue team has destroyed
3. blueFirstBlood: First kill of the game. 1 if the blue team did the first kill, 0 otherwise
4. blueKills: Number of enemies killed by the blue team
5. blueDeaths: Number of deaths (blue team)
6. blueAssists: Number of kill assists (blue team)
7. blueEliteMonsters: Number of elite monsters killed by the blue team (Dragons and Heralds)
8. blueDragons: Number of dragons killed by the blue team
9. blueHeralds: Number of heralds killed by the blue team
10. blueTowersDestroyed: Number of structures destroyed by the blue team (towers...)
11. blueTotalGold: Blue team total gold
12. blueAvgLevel: Blue team average champion level
13. blueTotalExperience: Blue team total experience
14. blueTotalMinionsKilled: Blue team total minions killed (CS)
15. blueTotalJungleMinionsKilled: Blue team total jungle monsters killed
16. blueGoldDiff: Blue team gold difference compared to the enemy team
17. blueExperienceDiff: Blue team experience difference compared to the enemy team
18. blueCSPerMin: Blue team CS (minions) per minute
19. blueGoldPerMin: Blue team gold per minute

1. redWardsPlaced:Number of warding totems placed by the red team on the map
2. redWardsDestroyed: Number of enemy warding totems the red team has destroyed
3. redFirstBlood: First kill of the game. 1 if the red team did the first kill, 0 otherwise
4. redKills: Number of enemies killed by the red team
5. redDeaths: Number of deaths (red team)
6. redAssists: Number of kill assists (red team)
7. redEliteMonsters: Number of elite monsters killed by the red team (Dragons and Heralds)
8. redDragons: Number of dragons killed by the red team
9. redHeralds: Number of heralds killed by the red team
10. redTowersDestroyed: Number of structures destroyed by the red team (towers...)
11. redTotalGold: Red team total gold
12. redAvgLevel: Red team average champion level
13. redTotalExperience: Red team total experience
14. redTotalMinionsKilled: Red team total minions killed (CS)
15. redTotalJungleMinionsKilled: Red team total jungle monsters killed
16. redGoldDiff: Red team gold difference compared to the enemy team
17. redExperienceDiff: Red team experience difference compared to the enemy team
18. redCSPerMin: Red team CS (minions) per minute
19. redGoldPerMin: Red team gold per minute

观察特征定义得：
1. blueFirstBlood 和 redFirstBlood 是互补关系，一个是0一个是1。留任意一个特征即可。❎
2. blueEliteMonsters = blueDragons + blueHeralds, 留后两个特征。✅
3. redEliteMonsters = redDragons + redHeralds, 留后两个特征。✅
4. blueTotalGold = blueGoldPerMin * 10 ✅
5. redTotalGold = redGoldPerMin * 10 ✅
6. blueTotalMinionsKilled = blueCSPerMin * 10 ✅
7. blueGoldDiff + redGoldDiff = 0 ✅
8. blueExperienceDiff + redExperienceDiff = 0 ✅
9. blueGoldDiff =  blueTotalGold - redTotalGold ✅
10. blueExperienceDiff =  blueTotalExperience - redTotalExperience ✅

In [69]:
print(data_df.iloc[0]) # 输出第一行数据
data_df.describe() # 每列特征的简单统计信息

blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueFirstBlood                      1.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalExperience             17039.0
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueGoldDiff                      643.0
blueExperienceDiff                 -8.0
blueCSPerMin                       19.5
blueGoldPerMin                   1721.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redFirstBlood                       0.0
redKills                            6.0
redDeaths                           9.0


,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
count,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,...,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000
mean,0.499038,22.288288,2.824881,0.504808,6.183925,6.137666,6.645106,0.549954,0.361980,0.187974,...,0.043021,16489.041401,6.925316,17961.730438,217.349226,51.313088,-14.414111,33.620306,21.734923,1648.904140
std,0.500024,18.019177,2.174998,0.500002,3.011028,2.933818,4.064520,0.625527,0.480597,0.390712,...,0.216900,1490.888406,0.305311,1198.583912,21.911668,10.027885,2453.349179,1920.370438,2.191167,149.088841
min,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,11212.000000,4.800000,10465.000000,107.000000,4.000000,-11467.000000,-8348.000000,10.700000,1121.200000
25%,0.000000,14.000000,1.000000,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,...,0.000000,15427.500000,6.800000,17209.500000,203.000000,44.000000,-1596.000000,-1212.000000,20.300000,1542.750000
50%,0.000000,16.000000,3.000000,1.000000,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,...,0.000000,16378.000000,7.000000,17974.000000,218.000000,51.000000,-14.000000,28.000000,21.800000,1637.800000
75%,1.000000,20.000000,4.000000,1.000000,8.000000,8.000000,9.000000,1.000000,1.000000,0.000000,...,0.000000,17418.500000,7.200000,18764.500000,233.000000,57.000000,1585.500000,1290.500000,23.300000,1741.850000
max,1.000000,250.000000,27.000000,1.000000,22.000000,22.000000,29.000000,2.000000,1.000000,1.000000,...,2.000000,22732.000000,8.200000,22269.000000,289.000000,92.000000,10830.000000,9333.000000,28.900000,2273.200000


### 增删特征
传统的机器学习模型大部分都是基于特征的，因此特征工程是机器学习中非常重要的一步。有时构造一个好的特征比改进一个模型带来的提升更大。这里简单展示一些特征处理的例子。首先，上面提到，特征列中有些特征信息是完全冗余的，会给模型带来不必要的计算量，可以去除。其次，相比于红蓝双方击杀、助攻的绝对值，可能双方击杀英雄的差值更能体现出当前对战的局势。因此，我们可以构造红蓝双方对应特征的差值。数据文件中已有的差值是金币差GoldDiff和经验差ExperienceDiff，实际上每个对应特征都可以构造这样的差值特征。

In [70]:
drop_features = ['blueGoldDiff', 'redGoldDiff', 
                 'blueExperienceDiff', 'redExperienceDiff', 
                 'blueCSPerMin', 'redCSPerMin', 
                 'blueGoldPerMin', 'redGoldPerMin'] # 需要舍去的特征列
#                  'blueEliteMonsters', 'redEliteMonsters'] 
df = data_df.drop(columns=drop_features) # 舍去特征列
info_names = [c[3:] for c in df.columns if c.startswith('red')] # 取出要作差值的特征名字（除去red前缀）
for info in info_names: # 对于每个特征名字
    df['br' + info] = df['blue' + info] - df['red' + info] # 构造一个新的特征，由蓝色特征减去红色特征，前缀为br
# 其中FirstBlood为首次击杀最多有一只队伍能获得，brFirstBlood=1为蓝，0为没有产生，-1为红
df = df.drop(columns=['blueFirstBlood', 'redFirstBlood']) # 原有的FirstBlood可删除

In [72]:
df.head()

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,...,brAssists,brEliteMonsters,brDragons,brHeralds,brTowersDestroyed,brTotalGold,brAvgLevel,brTotalExperience,brTotalMinionsKilled,brTotalJungleMinionsKilled
0,0,28,2,9,6,11,0,0,0,0,...,3,0,0,0,0,643,-0.2,-8,-2,-19
1,0,12,1,5,5,5,0,0,0,0,...,3,-2,-1,-1,-1,-2908,-0.2,-1173,-66,-9
2,0,15,0,7,11,4,1,1,0,0,...,-10,1,1,0,0,-1172,-0.4,-1033,-17,18
3,0,43,1,4,5,5,1,0,1,0,...,-5,1,0,1,0,-1321,0.0,-7,-34,8
4,0,75,4,6,6,6,0,0,0,0,...,-1,-1,-1,0,0,-1004,0.0,230,-15,-10


### 特征离散化
决策树ID3算法一般是基于离散特征的，本例中存在很多连续的数值特征，例如队伍金币。直接应用该算法每个值当作一个该特征的一个取值可能造成严重的过拟合，因此需要对特征进行离散化，即将一定范围内的值映射成一个值，例如对用户年龄特征，将0-10映射到0，11-18映射到1，19-25映射到2，25-30映射到3，等等类似，然后在决策树构建时使用映射后的值计算信息增益。

***本小节要求实现特征离散化，请补全相关代码***

In [73]:
discrete_df = df.copy() # 先复制一份数据(深拷贝)
for c in df.columns[1:]: # 遍历每一列特征，跳过标签列
    if len(np.unique(df[c]))>10:
        discrete_df[c] = pd.qcut(df[c],([0.,0.2,0.4,0.6,0.8,1.]),labels=[0,1,2,3,4])
#         continue
    else:
        continue

discrete_df.head()

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,...,brAssists,brEliteMonsters,brDragons,brHeralds,brTowersDestroyed,brTotalGold,brAvgLevel,brTotalExperience,brTotalMinionsKilled,brTotalJungleMinionsKilled
0,0,4,1,3,2,4,0,0,0,0,...,3,0,0,0,0,3,1,2,2,0
1,0,0,0,1,1,1,0,0,0,0,...,3,-2,-1,-1,-1,0,1,1,0,1
2,0,1,0,2,4,1,1,1,0,0,...,0,1,1,0,0,1,1,1,1,4
3,0,4,0,0,1,1,1,0,1,0,...,0,1,0,1,0,1,2,2,0,3
4,0,4,3,2,2,2,0,0,0,0,...,1,-1,-1,0,0,1,2,2,1,1


### 数据集准备

In [74]:
all_y = discrete_df['blueWins'].values # 所有标签数据
feature_names = discrete_df.columns[1:] # 所有特征的名称
all_x = discrete_df[feature_names].values # 所有原始特征值，pandas的DataFrame.values取出为numpy的array矩阵

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=0.2, random_state=RANDOM_SEED)
all_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息

((9879,), (9879, 43), (7903, 43), (1976, 43), (7903,), (1976,))

###  决策树模型的实现
***本小节要求实现决策树模型，请补全算法代码***

In [75]:
class Node:

    def __init__(self, x, y, impurity_method, dimension = -1, value = -1):
        self.x = x
        self.y = y
        self.dimension = dimension
        self.value = value
        self.xlen = len(x)
        self.ylen = len(y)
        self.impurity_method = impurity_method
        self.left = None
        self.right = None
    
    def impurity_entropy(self):

        counter = Counter(self.y)
        ent = 0.0
        for num in counter.values():
            p = num / len(self.y)
            ent += -p * np.log2(p)
        return ent
    
    def impurity_gini(self):

        counter = Counter(self.y)
        gini = 0
        sum = 0
        for num in counter.values():
            p = num / len(self.y)
            sum += p ** 2
        gini = 1 - sum
        return gini
    
    def impurity(self):
        if self.impurity_method == 'entropy':
            return self.impurity_entropy()
        elif self.impurity_method == 'gini':
            return self.impurity_gini()
    
    def best_split(self):

        best_impurity = float('inf')
        best_dimension = -1
        best_value = -1

        for d in range(self.x.shape[1]):
            sorted_index = np.argsort(self.x[:, d])
            for i in range(1, len(self.x)):
                if self.x[sorted_index[i], d] != self.x[sorted_index[i - 1], d]:
                    value = (self.x[sorted_index[i], d] + self.x[sorted_index[i - 1], d]) / 2
                    index_left = (self.x[:, d] <= value)
                    index_right = (self.x[:, d] > value)
                    x_left, x_right, y_left, y_right = self.x[index_left], self.x[index_right], self.y[index_left], self.y[index_right]
                    node_left = Node(x_left, y_left, self.impurity_method)
                    node_right = Node(x_right, y_right, self.impurity_method)

                    p_left = len(x_left) / len(self.x)
                    p_right = len(x_right) / len(self.x)
                        
                    impurity = p_left * node_left.impurity() + p_right * node_right.impurity()
                    if impurity < best_impurity:
                        best_impurity = impurity
                        best_dimension = d
                        best_value = value
        return best_impurity, best_dimension, best_value
    

In [76]:
# 定义决策树类
class DecisionTree(object):
    def __init__(self, classes, features, 
                 max_depth=10, min_samples_split=10, impurity_t = 'entropy'):
        '''
        传入一些可能用到的模型参数，也可能不会用到
        classes表示模型分类总共有几类
        features是每个特征的名字，也方便查询总的特征数
        max_depth表示构建决策树时的最大深度
        min_samples_split表示构建决策树分裂节点时，如果到达该节点的样本数小于该值则不再分裂 len(x)<...
        impurity_t表示计算混杂度（不纯度）的计算方式，例如entropy或gini
        '''  
        self.classes = classes
        self.features = features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_t = impurity_t
        self.root = None # 定义根节点，未训练时为空

    def fit(self, x,y):
        assert len(self.features) == len(x[0]) # 输入数据的特征数目应该和模型定义时的特征数目相同
        '''
        训练模型
        feature为二维numpy（n*m）数组，每行表示一个样本，有m个特征
        label为一维numpy（n）数组，表示每个样本的分类标签
        
        提示：一种可能的实现方式为
        self.root = self.expand_node(feature, label, depth=1) # 从根节点开始分裂，模型记录根节点
        '''

        def expand_node(x, y, depth = 0):

            node = Node(x,y,impurity_method = self.impurity_t)

            #开始递归
            if depth >= self.max_depth:  #if max depth = current depth -->stop
                return node
            elif node.xlen <= self.min_samples_split: 
                return node
            else:
                #计算当前节点的impurity
                impurity_base = node.impurity()
                if impurity_base == 0:
                    return node
                else:
                    best_impurity, best_dimension, best_value = node.best_split()
                    if best_impurity >= impurity_base:
                        return node
                    else:
                        node.dimension = best_dimension
                        node.value = best_value
                        index_left = (x[:, best_dimension] <= best_value)
                        index_right = (x[:, best_dimension] > best_value)
                        x_l, x_r, y_l, y_r, depth = x[index_left], x[index_right], y[index_left], y[index_right], depth+1
                        node.left = expand_node(x_l, y_l, depth)
                        node.right = expand_node(x_r, y_r, depth)
                        return node
    
        # 从跟节点开始分裂
        self.root = expand_node(x, y)

        return self

        
    
    def predict(self, feature):
        assert len(feature.shape) == 1 or len(feature.shape) == 2 # 只能是1维或2维
        '''
        预测
        输入feature可以是一个一维numpy数组也可以是一个二维numpy数组
        如果是一维numpy（m）数组则是一个样本，包含m个特征，返回一个类别值
        如果是二维numpy（n*m）数组则表示n个样本，每个样本包含m个特征，返回一个numpy一维数组
        
        traverse_node()预测时遍历节点，考虑不同情况
        1. 已经到达叶节点，则返回分类结果
        2. 该特征取值在训练集中未出现过
        3. 依据特征取值进入相应子节点，递归调用traverse_node
        
        提示：一种可能的实现方式为
        if len(feature.shape) == 1: # 如果是一个样本
            return self.traverse_node(self.root, feature) # 从根节点开始路由
        return np.array([self.traverse_node(self.root, f) for f in feature]) # 如果是很多个样本
        
        '''
        
        def traverse_node(x_data, node):

            if x_data[node.dimension] <= node.value and node.left:
                pred = traverse_node(x_data, node.left)
            elif x_data[node.dimension] > node.value and node.right:
                pred = traverse_node(x_data, node.right)
            else:
                counter = Counter(node.y)
                pred = counter.most_common(1)[0][0]
            return pred

        y_predict = []
        for data in feature:
            y_pred = traverse_node(data, self.root)
            y_predict.append(y_pred)
        return np.array(y_predict)

In [77]:
def fit_predict(method):
    
    print('Results when using', method, 'for the impurity measurement')
    # 定义决策树模型，传入算法参数
    DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=5, min_samples_split=10, impurity_t=method)
    DT.fit(x_train, y_train) # 在训练集上训练
    p_test = DT.predict(x_test) # 在测试集上预测，获得预测值
    print(p_test) # 输出预测值
    test_acc = accuracy_score(p_test, y_test) # 将测试预测值与测试集标签对比获得准确率
    print('accuracy: {:.4f}'.format(test_acc)) # 输出准确率# 定义决策树模型，传入算法参数

In [78]:
fit_predict('entropy')
print('\n')
fit_predict('gini')

Results when using entropy for the impurity measurement
[0 1 0 ... 0 1 1]
accuracy: 0.7272


Results when using gini for the impurity measurement
[0 1 0 ... 0 1 1]
accuracy: 0.7262
